In [1]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper, DuckDuckGoSearchAPIWrapper
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
template = """
    Bạn là một trợ lý ảo giúp người dùng tìm kiếm thông tin trên Wikipedia và DuckDuckGo. Hãy trả lời các câu hỏi của người dùng bằng tiếng Việt.

    **Phương pháp suy ra query:**
    1. **Xác định ý định câu hỏi:**
       - Nhận diện các câu hỏi yêu cầu thông tin (ví dụ: "là gì", "cho tôi biết", "giúp tôi tìm", "tìm thông tin về", "cho tôi hỏi về", v.v.).

    2. **Trích xuất nội dung chính:**
       - Nếu câu hỏi chứa các cụm từ chỉ định, trích xuất nội dung đứng sau các cụm từ đó làm **query**.
       - Nếu không có cụm từ chỉ định rõ ràng, xác định danh từ chính hoặc chủ đề trung tâm của câu hỏi làm **query**.
       - Xử lý linh hoạt các biến thể câu hỏi bằng cách nhận diện cấu trúc câu và các từ khóa quan trọng.

    Ví dụ:
    - "Bạn hãy cho mình biết thông tin về Albert Einstein." → query = "Albert Einstein"
    - "Python là gì?" → query = "Python"
    - "Ai phát minh ra bóng đèn?" → query = "người phát minh ra bóng đèn"

    **Kịch bản:**
    1. Trợ lý trả lời: "Mình là một trợ lý ảo giúp người dùng tìm kiếm thông tin trên Wikipedia và DuckDuckGo."
    2. Người dùng hỏi: "Bạn có thể giúp tôi tìm thông tin về {input_message} không?"
    3. Trợ lý trả lời: "\nDạ, mình sẽ giúp bạn tìm thông tin về {input_message}."
    4. Trợ lý tìm kiếm thông tin về {input_message} trên Wikipedia.
    Nếu tìm thấy thông tin trên Wikipedia:
        5. Trợ lý trả lời: "\nĐây là thông tin về {input_message}: ..."
        6. Người dùng hỏi: "Cảm ơn bạn."
        7. Trợ lý trả lời: "Không có gì, bạn cứ hỏi nếu cần giúp đỡ."

    Nếu không tìm thấy thông tin trên Wikipedia:
        5. Trợ lý trả lời: "\nXin lỗi, mình không tìm thấy thông tin về {input_message} trên Wikipedia. Mình sẽ thử tìm kiếm trên DuckDuckGo."
        6. Trợ lý tìm kiếm thông tin về {input_message} trên DuckDuckGo.
        7. Trợ lý trả lời: "\nĐây là thông tin về {input_message}: ..."
        8. Người dùng hỏi: "Cảm ơn bạn."
        9. Trợ lý trả lời: "Không có gì, bạn cứ hỏi nếu cần giúp đỡ."

    **Lưu ý:**
    - Linh hoạt nhận diện và xử lý các biến thể ngôn ngữ trong câu hỏi.
    - Ưu tiên kết quả từ Wikipedia trước, sau đó mới tìm trên DuckDuckGo nếu cần thiết.
"""

In [3]:
prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        ("placeholder", "{chat_history}"),
        ("human", "{input_message}"),
        ("placeholder", "{agent_scratchpad}"),
])

In [4]:
@tool(description="Search for information on Wikipedia in Vietnamese")
def search_wikipedia(query: str) -> str:
    """
    Searches for information on Wikipedia using the given query in Vietnamese.
    
    Args:
        query (str): The search query string.
    
    Returns:
        str: Search results from Wikipedia in Vietnamese.
    """
    # Tạo đối tượng Wikipedia API với ngôn ngữ tiếng Việt
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(lang="vi"))

    # Thực thi truy vấn và trả về kết quả dưới dạng chuỗi
    result = wikipedia.invoke(query)
    return result if result else "Không tìm thấy thông tin trên Wikipedia."

@tool(description="Search for information on DuckDuckGo in Vietnamese")
def search_duckduckgo(query: str) -> str:
    """
    Searches for information on DuckDuckGo using the given query.
    
    Args:
        query (str): The search query string.
    
    Returns:
        str: Search results from DuckDuckGo in Vietnamese.
    """
    wrapper = DuckDuckGoSearchAPIWrapper(region="vn", time="d", max_results=1)
    search = DuckDuckGoSearchRun(api_wrapper=wrapper)

    result = search.invoke(query)
    return result if result else "Không tìm thấy thông tin trên DuckDuckGo."

@tool(description="Search for information on both Wikipedia and DuckDuckGo in Vietnamese")
def search(query: str) -> str:
    """
    Searches for information on both Wikipedia and DuckDuckGo using the given query.
    
    Args:
        query (str): The search query string.
    
    Returns:
        str: Combined search results.
    """
    wikipedia_result = search_wikipedia(query)
    if "Không tìm thấy thông tin" not in wikipedia_result:
        return wikipedia_result
    else:
        return search_duckduckgo(query)

In [20]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", max_tokens=100, temperature=0.5, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0)

tools = [search_wikipedia, search_duckduckgo]

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [21]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

agent_with_history = RunnableWithMessageHistory(
    agent_executor, 
    get_session_history,
    input_messages_key="input_message",
    history_messages_key="chat_history",
)

In [22]:
agent_with_history.invoke(
    {"input_message": "Cho mình biết thông tin về Việt Nam."},
    config={"configurable": {"session_id": "ss1"}},
)



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Việt Nam'}`
responded: Mình là một trợ lý ảo giúp người dùng tìm kiếm thông tin trên Wikipedia và DuckDuckGo.
Dạ, mình sẽ giúp bạn tìm thông tin về Việt Nam.

Page: Việt Nam
Summary: Việt Nam, quốc hiệu là Cộng hòa xã hội chủ nghĩa Việt Nam, là một quốc gia nằm ở cực Đông của bán đảo Đông Dương thuộc khu vực Đông Nam Á, giáp với Lào, Campuchia, Trung Quốc, biển Đông và vịnh Thái Lan.
Lãnh thổ Việt Nam xuất hiện con người sinh sống từ thời đại đồ đá cũ, khởi đầu với các nhà nước Văn Lang, Âu Lạc. Âu Lạc bị nhà Triệu ở phương Bắc thôn tính vào đầu thế kỷ thứ 2 TCN sau đó là thời kỳ Bắc thuộc kéo dài hơn một thiên niên kỷ. Chế độ quân chủ độc lập được tái lập sau chiến thắng của Ngô Quyền trước nhà Nam Hán, mở đường cho các triều đại độc lập kế tục, nhiều lần chiến thắng trước các cuộc chiến tranh xâm lược từ phương Bắc đồng thời dần mở rộng về phía nam. Thời kỳ Bắc thuộc cuối cùng kết thúc sau chiến th

{'input_message': 'Cho mình biết thông tin về Việt Nam.',
 'chat_history': [],
 'output': 'Đây là thông tin về Việt Nam:\n\nViệt Nam, quốc hiệu là Cộng hòa xã hội chủ nghĩa Việt Nam, là một quốc gia nằm ở cực Đông của bán đảo Đông Dương thuộc khu vực Đông Nam Á, giáp với Lào, Campuchia, Trung Quốc, biển Đông và vịnh Thái Lan.\nLãnh thổ Việt Nam xuất hiện con người sinh sống từ thời đại đồ đá cũ, khởi đầu với các nhà nước Văn Lang, Âu Lạc.'}

In [24]:
agent_with_history.invoke(
    {"input_message": "Vậy thủ đô của Việt Nam là gì?"},
    config={"configurable": {"session_id"   : "ss1"}}
)



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Thủ đô của Việt Nam'}`
responded: Dạ, mình sẽ giúp bạn tìm thông tin về Vậy thủ đô của Việt Nam là gì?.

Page: Thủ đô Việt Nam
Summary: Thủ đô Việt Nam hiện nay là thành phố Hà Nội. Sau đây là danh sách các kinh đô/thủ đô – hiểu theo nghĩa rộng – là các trung tâm chính trị của chính thể nhà nước trong lịch sử Việt Nam, và cả của các vương quốc cổ/cựu quốc gia từng tồn tại trên lãnh thổ Việt Nam ngày nay.

Page: Việt Nam
Summary: Việt Nam, quốc hiệu là Cộng hòa xã hội chủ nghĩa Việt Nam, là một quốc gia nằm ở cực Đông của bán đảo Đông Dương thuộc khu vực Đông Nam Á, giáp với Lào, Campuchia, Trung Quốc, biển Đông và vịnh Thái Lan.
Lãnh thổ Việt Nam xuất hiện con người sinh sống từ thời đại đồ đá cũ, khởi đầu với các nhà nước Văn Lang, Âu Lạc. Âu Lạc bị nhà Triệu ở phương Bắc thôn tính vào đầu thế kỷ thứ 2 TCN sau đó là thời kỳ Bắc thuộc kéo dài hơn một thiên niên kỷ. Chế độ quân chủ độc lập được tái lậ

{'input_message': 'Vậy thủ đô của Việt Nam là gì?',
 'chat_history': [HumanMessage(content='Cho mình biết thông tin về Việt Nam.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Đây là thông tin về Việt Nam:\n\nViệt Nam, quốc hiệu là Cộng hòa xã hội chủ nghĩa Việt Nam, là một quốc gia nằm ở cực Đông của bán đảo Đông Dương thuộc khu vực Đông Nam Á, giáp với Lào, Campuchia, Trung Quốc, biển Đông và vịnh Thái Lan.\nLãnh thổ Việt Nam xuất hiện con người sinh sống từ thời đại đồ đá cũ, khởi đầu với các nhà nước Văn Lang, Âu Lạc.', additional_kwargs={}, response_metadata={})],
 'output': 'Thủ đô của Việt Nam hiện nay là thành phố Hà Nội.'}

In [25]:
agent_with_history.invoke(
    {"input_message": "Cho mình thông tin về những người tham gia chương trình anh trai 'Say Hi'."},
    config={"configurable": {"session_id": "ss1"}}
)



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'những người tham gia chương trình anh trai "Say Hi"'}`
responded: Dạ, mình sẽ giúp bạn tìm thông tin về những người tham gia chương trình anh trai 'Say Hi'.

Page: Anh trai "say hi"
Summary: Anh trai "say hi" là một chương trình truyền hình thực tế về âm nhạc được phát sóng trên kênh HTV2 - Vie Channel, ON Vie Giải Trí và ứng dụng VieON từ ngày 15 tháng 6 năm 2024. Chương trình có sự tham gia của 30 "anh trai" là những nam nghệ sĩ đang hoạt động trong lĩnh vực nghệ thuật.

Page: Anh Tú (sinh 1992)
Summary: Nguyễn Anh Tú (sinh ngày 30 tháng 5 năm 1992), thường được biết đến với nghệ danh Anh Tú, là một nam ca sĩ người Việt Nam.

Page: Anh trai "say hi" (mùa 1)
Summary: Mùa đầu tiên của chương trình Anh trai "say hi" được phát sóng trên kênh truyền hình HTV2 - Vie Channel, ON Vie Giải Trí và ứng dụng VieON từ ngày 15 tháng 6 đến ngày 14 tháng 9 năm 2024.
Người chiến thắng của mùa này là HIEUTHUHAI, và 

{'input_message': "Cho mình thông tin về những người tham gia chương trình anh trai 'Say Hi'.",
 'chat_history': [HumanMessage(content='Cho mình biết thông tin về Việt Nam.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Đây là thông tin về Việt Nam:\n\nViệt Nam, quốc hiệu là Cộng hòa xã hội chủ nghĩa Việt Nam, là một quốc gia nằm ở cực Đông của bán đảo Đông Dương thuộc khu vực Đông Nam Á, giáp với Lào, Campuchia, Trung Quốc, biển Đông và vịnh Thái Lan.\nLãnh thổ Việt Nam xuất hiện con người sinh sống từ thời đại đồ đá cũ, khởi đầu với các nhà nước Văn Lang, Âu Lạc.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Vậy thủ đô của Việt Nam là gì?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Thủ đô của Việt Nam hiện nay là thành phố Hà Nội.', additional_kwargs={}, response_metadata={})],
 'output': 'Dạ, mình sẽ giúp bạn tìm thông tin về những người tham gia chương trình anh trai \'Say Hi\'.\n\nĐây là thông tin về những người

In [26]:
agent_with_history.invoke(
    {"input_message": "Cảm ơn bạn."},
    config={"configurable": {"session_id": "ss1"}}
)



> Entering new AgentExecutor chain...
Không có gì, bạn cứ hỏi nếu cần giúp đỡ.

> Finished chain.


{'input_message': 'Cảm ơn bạn.',
 'chat_history': [HumanMessage(content='Cho mình biết thông tin về Việt Nam.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Đây là thông tin về Việt Nam:\n\nViệt Nam, quốc hiệu là Cộng hòa xã hội chủ nghĩa Việt Nam, là một quốc gia nằm ở cực Đông của bán đảo Đông Dương thuộc khu vực Đông Nam Á, giáp với Lào, Campuchia, Trung Quốc, biển Đông và vịnh Thái Lan.\nLãnh thổ Việt Nam xuất hiện con người sinh sống từ thời đại đồ đá cũ, khởi đầu với các nhà nước Văn Lang, Âu Lạc.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Vậy thủ đô của Việt Nam là gì?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Thủ đô của Việt Nam hiện nay là thành phố Hà Nội.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="Cho mình thông tin về những người tham gia chương trình anh trai 'Say Hi'.", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Dạ, mình sẽ giúp bạn tìm thông tin v